In [2]:
import matplotlib
matplotlib.use('Agg')
# import sys
# sys.path.append("/home/ec2-user/anaconda3/external/fastai/")
# sys.path.append('/scratch/arka/miniconda3/external/fastai/')


from tqdm import tqdm
tqdm.monitor_interval = 0

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import json
import pandas as pd
from sklearn.metrics import *

In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
PATH = Path("/scratch/arka/miniconda3/external/fastai/courses/dl2/data/cifar10/")
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [5]:
def total_model_params(summary):
    total_params = 0
    for key,value in summary.items():
        total_params+=int(summary[key]['nb_params'])
    print("Total parameters in the model :"+str(total_params))

In [6]:
bs=128
sz=32

In [7]:
class depthwise_block(nn.Module):
    def __init__(self, in_c, out_c, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_c)
        self.conv1 = nn.Conv2d(in_c, in_c, kernel_size=3, 
                               stride=stride, padding=1,
                               groups=in_c, bias=False)
        self.conv2 = nn.Conv2d(in_c, out_c, kernel_size=1,
                              stride=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_c)
    
    def forward(self, inp):
        out = F.relu(self.bn1(self.conv1(inp)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out

In [8]:
class mblnetv1(nn.Module):
    def __init__(self, block, inc_list, inc_scale, num_blocks_list, stride_list, num_classes):
        super().__init__()
        self.num_blocks = len(num_blocks_list)
        inc_list1 = [o//inc_scale for o in inc_list]
        self.in_planes = inc_list1[0]
        self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_planes)
        
        lyrs = []
        for inc, nb, strl in zip(inc_list1[1:], num_blocks_list, stride_list):
            lyrs.append(self._make_layer(block, inc, nb, strl))
            
        self.lyrs = nn.Sequential(*lyrs)
        self.linear = nn.Linear(inc_list1[-1], num_classes)
        
    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, inp):
        out = F.relu(self.bn1(self.conv1(inp)))
        out = self.lyrs(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return F.log_softmax(out, dim=-1)


In [9]:
md_mbl = mblnetv1(depthwise_block, 
              inc_list=[64, 64, 128, 256], 
              inc_scale = 1, 
              num_blocks_list=[2, 2, 2], 
              stride_list=[1, 2, 2], 
              num_classes=10)

In [ ]:
# md_mbl2 = mblnetv1(depthwise_block, 
#               inc_list=[32, 64, 128, 128, 256, 256, 512, 512, 1024, 1024], 
#               inc_scale = 1, 
#               num_blocks_list=[1, 1, 1, 1, 1, 1, 4, 1, 1], 
#               stride_list=[1, 2, 1, 2, 1, 2, 1, 2, 1], 
#               num_classes=10)

In [ ]:
!ls {PATH / 'train'}

In [9]:
data = get_data(sz, bs)

In [10]:
learn = ConvLearner.from_model_data(md_mbl, data)

In [11]:
learn.summary()

OrderedDict([('Conv2d-1',
              OrderedDict([('input_shape', [-1, 3, 32, 32]),
                           ('output_shape', [-1, 64, 32, 32]),
                           ('trainable', True),
                           ('nb_params', 1728)])),
             ('BatchNorm2d-2',
              OrderedDict([('input_shape', [-1, 64, 32, 32]),
                           ('output_shape', [-1, 64, 32, 32]),
                           ('trainable', True),
                           ('nb_params', 128)])),
             ('Conv2d-3',
              OrderedDict([('input_shape', [-1, 64, 32, 32]),
                           ('output_shape', [-1, 64, 32, 32]),
                           ('trainable', True),
                           ('nb_params', 576)])),
             ('BatchNorm2d-4',
              OrderedDict([('input_shape', [-1, 64, 32, 32]),
                           ('output_shape', [-1, 64, 32, 32]),
                           ('trainable', True),
                           ('nb_params', 128

In [12]:
total_model_params(learn.summary())

Total parameters in the model :145034


In [ ]:
learn.fit(1e-2, 4, cycle_len=10, best_save_name='mobilenetv1_1', metrics=[accuracy])

In [13]:
learn.load('mobilenetv1_1')

In [14]:
learn.fit(1e-2, 2, cycle_len=10, cycle_mult=2, best_save_name='mobilenetv1_1', metrics=[accuracy])

epoch      trn_loss   val_loss   accuracy                    
    0      0.37439    0.460946   0.8474    
    1      0.375303   0.48399    0.838                       
    2      0.33792    0.43484    0.8528                      
    3      0.321283   0.411093   0.8623                      
    4      0.294801   0.403581   0.8629                      
    5      0.283703   0.385807   0.8707                      
    6      0.260451   0.374656   0.8744                      
    7      0.238994   0.367168   0.878                       
    8      0.23828    0.366056   0.8754                      
    9      0.235172   0.365442   0.8764                      
    10     0.329295   0.442654   0.854                       
    11     0.336549   0.453259   0.8509                      
    12     0.313774   0.466541   0.849                       
    13     0.313518   0.447262   0.8531                      
    14     0.310718   0.436877   0.8565                      
    15     0.286829   0.40

[array([0.35745]), 0.8838]